In [17]:
import pulp
import numpy as np
from random import randint, random
np.set_printoptions(suppress=True)

In [10]:
M = 10**4
ZERO_ROW_CHANCE = 0.05
ZERO_COL_CHANCE = 0.05
M_CHANCE = 0.1
# arbitrary ^

def generate_problem(N):
    s = np.random.randint(0, 131, N) # supply randomisation slightly higher on average to make demand easier to work with

    # the idea is to randomly generate random demands and put the remaining supply into the last node
    # the supply is meant to be larger on average to make this easier,
    # but in case of demand outpacing supply before the end of the loop, we regenerate the demand array
    while True:
        d = np.random.randint(0, 101, N)
        supply_left = s.sum() - d[:-1].sum()
        if supply_left < 0:
            continue
        d[-1] = supply_left
        break

    # the cost matrix is just random values with occasional M's
    c_matrix = np.random.randint(0, 31, (N, N))
    m_mask = np.random.rand(N, N) < M_CHANCE
    c_matrix[m_mask] = M

    # very rare 0 columns and rows
    if random() < ZERO_ROW_CHANCE:
        c_matrix[randint(0, N-1), :] = 0
    if random() < ZERO_COL_CHANCE:
        c_matrix[:, randint(0, N-1)] = 0
    return s, d, c_matrix

In [20]:
s, d, c_matrix = generate_problem(5) # example N

def pulp_solve(s, d, c_matrix):
    model = pulp.LpProblem("Transportation validation", pulp.LpMinimize)
    x = pulp.LpVariable.dicts("x", (range(c_matrix.shape[0]), range(c_matrix.shape[0])), lowBound=0)

    model += pulp.lpSum(c_matrix[i, j] * x[i][j] for i in range(c_matrix.shape[0]) for j in range(c_matrix.shape[0]))

    for i in range(len(c_matrix)):
        model += pulp.lpSum(x[i][j] for j in range(c_matrix.shape[0])) == s[i]
        model += pulp.lpSum(x[j][i] for j in range(c_matrix.shape[0])) == d[i]

    model.solve()
    return model.objective.value()

# print(s)
# print(d)
# print_matrix(matrix)
print(pulp_solve(s, d, c_matrix))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/silver/tf/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/c84cd6a3b49d4034a3cff4e6f38f2a96-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/c84cd6a3b49d4034a3cff4e6f38f2a96-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 15 COLUMNS
At line 91 RHS
At line 102 BOUNDS
At line 103 ENDATA
Problem MODEL has 10 rows, 25 columns and 50 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 10 (0) rows, 25 (0) columns and 50 (0) elements
0  Obj 0 Primal inf 516 (10)
10  Obj 2671
Optimal - objective value 2671
Optimal objective 2671 - 10 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

2671.0


In [ ]:
def solve(s, d, c_matrix):
    basis = np.zeros_like(c_matrix, np.bool)
    matrix = np.zeros_like(c_matrix)
    supply_left = s.copy()
    demand_left = d.copy()
    n = s.shape[0]
    # first run
    i = 0
    j = 0
    while i < n and j < n:
        deduction = min(supply_left[i], demand_left[j])
        matrix[i, j] = deduction
        basis[i, j] = True
        supply_left[i] -= deduction
        demand_left[j] -= deduction
        if supply_left[i] == 0:
            i += 1
        elif demand_left[j] == 0:
            j += 1
        if not supply_left.sum() > 0 and not demand_left.sum() > 0:
            break
    # print(basis)
    # print(matrix)
    # main loop (while not optimal)
    while True:
        # calculating u and v
        u = np.full(n, np.nan)
        v = np.full(n, np.nan)
        A = np.zeros((n*2, basis.sum() + 1), np.int32)
        b = np.zeros((basis.sum() + 1), np.int32)
        print(A.shape, b.shape)
        A[0, 0] = 1
        b[0] = 0
        basis_cnt = 1
        for i in range(n):
            for j in range(n):
                if basis[i, j]:
                    # u[i] + v[i] = c_matrix[i, j]
                    A[basis_cnt, i] = 1 # u
                    A[basis_cnt, n + j] = 1 # v
                    # ^ TODO: IndexError: index 19 is out of bounds for axis 1 with size 19 (super rare)
                    b[basis_cnt] = c_matrix[i, j] # c
                    basis_cnt += 1
        x = np.linalg.solve(A, b).astype(np.int32)
        u = x[:n]
        v = x[n:]
        # print(u)
        # print(v)
        # calculating all of the coefficients
        for i in range(n):
            for j in range(n):
                if not basis[i, j]:
                    matrix[i, j] = c_matrix[i, j] - u[i] - v[j]
                if c_matrix[i, j] == M: # M is supposed to stay as M
                    matrix[i, j] = M
        
        print(basis)
        print(matrix)
        # finding a cycle
        # in case of multiple minimums pick the first one
        pivot = np.argwhere(matrix == matrix.min())[:1]

        


        return # temp
        if np.all(matrix >= 0):
            break

    



for _ in range(1):
    solve(*generate_problem(10))

(20, 20) (20,)
[[ True False False False False False False False False False]
 [ True  True False False False False False False False False]
 [False  True  True False False False False False False False]
 [False False  True False False False False False False False]
 [False False  True  True  True False False False False False]
 [False False False False  True  True False False False False]
 [False False False False False  True  True  True False False]
 [False False False False False False False  True  True False]
 [False False False False False False False False  True False]
 [False False False False False False False False  True  True]]
[[    57    -16     -6    -19 -10007  -9998    -13    -17  -9988  -9998]
 [    40     35      3    -29  -9991  -9982     -4      5  -9981  -9998]
 [    12     27     49    -11  10000  -9972  10000      0  -9996  -9989]
 [    29     -5      4    -16  -9998  -9990     -5     15  -9984  -9987]
 [    17      2     41     21  10000  -9973  10000     -3  -99